In [1]:
import pandas as pd

## Load Alarms with Lavori Programmati

In [11]:
mob = pd.read_parquet("alarms datasets/mob/[new]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [2]:
adsl = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")
pdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet")
ptn = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")
sdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")

In [12]:
mob = mob[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
adsl = adsl[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
pdh = pdh[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
ptn = ptn[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
sdh = sdh[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]

In [13]:
alarms_slogan_lavori_programmati = pd.concat([mob, adsl, pdh, ptn, sdh])

In [14]:
alarms_slogan_lavori_programmati.to_parquet("alarms datasets/alarms_slogan_lavori_programmati.parquet")

## Merge Clusters and Alarms with Lavori Programmati

In [67]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")

In [68]:
clusters_with_lavori_programmati = pd.merge(clusters, alarms_slogan_lavori_programmati, on="alarm_id", how="left")

In [69]:
clusters_with_lavori_programmati = clusters_with_lavori_programmati[
    ["alarm_id", "cluster_id2", "std_probable_cause_no", "lp_type"]
]

In [94]:
filtered_df = clusters_with_lavori_programmati[clusters_with_lavori_programmati['lp_type'].apply(lambda x: x != 'None')]
filtered_df

,alarm_id,cluster_id2,std_probable_cause_no,lp_type
71358,COS1P;137150415,202301090504_2,loss of signal los,['APPARATI']
71359,COS1P;137150413,202301090504_2,loss of frame lof,['APPARATI']
71361,COS1P;137150372,202301090504_2,loss of signal los,['APPARATI']
71362,COS1P;137150380,202301090506_4,loss of signal los,['APPARATI']
71363,COS1P;137150381,202301090506_4,loss of signal los,['APPARATI']
...,...,...,...,...
8450196,EFMTX;65838f67373d2e0001c8d1d0,202312210208_7,loss of frame lof,['CAVO']
8450199,EFMTX;65838ef188b9a400019bf349,202312210200_6,loss of frame lof,['CAVO']
8450200,EFMTX;65838f2b40918a0001130ca9,202312210200_6,loss of frame lof,['CAVO']
8450203,EFMTX;65838f67373d2e0001c8d1d0,202312210200_6,loss of frame lof,['CAVO']


In [90]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,lp_type
0,COS1P;136885076,202301010000_16,apparato isolato,None
1,COS1P;136885115,202301010000_16,apparato isolato,None
2,COS1P;136884642,202301010000_25,apparato isolato,None
3,COS1P;136884882,202301010000_25,apparato isolato,None
4,COS1P;136884883,202301010000_25,apparato isolato,None
...,...,...,...,...
8607319,EFM;6591f07dadefac0001744e96,202312312354_7,base station connectivity problem,None
8607320,EFM;6591f08739628d00013784db,202312312354_7,base station connectivity problem,None
8607321,EFM;6591f08cadefac0001744ea1,202312312354_7,base station connectivity problem,None
8607322,EFMTX;6591ee0801021b00010facb4,202312312340_13,fermo totale apparato,None


In [77]:
unique_lp_type_values = clusters_with_lavori_programmati['lp_type'].apply(lambda x: str(x)).unique()
unique_lp_type_values

array(['None', "['APPARATI']", "['CAVO']", "['DWDM/PTN']",
       "['REIX/UPGRADE']", "['CAMBI RELEASE']", "['CAVO' 'DWDM/PTN']",
       "['CAVO' 'APPARATI']"], dtype=object)

In [76]:
clusters_with_lavori_programmati['lp_type'] = clusters_with_lavori_programmati['lp_type'].replace([None], ['None'])

In [78]:
clusters_grouped = clusters_with_lavori_programmati.groupby(["cluster_id2"])

In [83]:
def check_cavo_alarm(group):
    return int(any('CAVO' in sublist for sublist in group['lp_type'].values if sublist is not None))

new_df = clusters_grouped.apply(lambda group: pd.Series({
    'cluster_id2': group.name,
    'GUASTO CAVO': check_cavo_alarm(group)
})).reset_index(drop=True)

C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_13796\1027940921.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_df = clusters_grouped.apply(lambda group: pd.Series({


In [85]:
new_df.sort_values(by='GUASTO CAVO', ascending=False)

,cluster_id2,GUASTO CAVO
755904,202311230212_1,1
625667,202310120512_1,1
237116,202305240122_12,1
214405,202305120432_11,1
714004,202311090118_21,1
...,...,...
288987,202306151940_15,0
288988,202306151940_8,0
288989,202306151942_4,0
288990,202306151944_2,0


In [87]:
guasto_cavo_count = new_df['GUASTO CAVO'].sum()
guasto_cavo_count

3447